In [2]:
import json
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches
import numpy as np

In [3]:
import json
import os
import numpy as np

def convert_coco_to_yolo(coco_annotations_path, images_dir, output_dir):
    with open(coco_annotations_path, 'r') as f:
        data = json.load(f)

    images = {image['id']: image for image in data['images']}
    annotations = data['annotations']
    categories = {category['id']: category for category in data['categories']}

    category_name_to_id = {category['name']: idx for idx, category in enumerate(categories.values())}

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)

    for annotation in annotations:
        image_id = annotation['image_id']
        image_info = images[image_id]
        image_filename = image_info['file_name']
        image_width = image_info['width']
        image_height = image_info['height']
        image_path = os.path.join(images_dir, image_filename)

        if not os.path.exists(image_path):
            continue

        output_image_path = os.path.join(output_dir, 'images', image_filename)
        output_label_path = os.path.join(output_dir, 'labels', os.path.splitext(image_filename)[0] + '.txt')

        os.system(f'cp "{image_path}" "{output_image_path}"')

        with open(output_label_path, 'a') as f:
            category_name = categories[annotation['category_id']]['name']
            category_id = category_name_to_id[category_name]
            
            # Normalize bounding box coordinates
            bbox = annotation['bbox']
            bbox[0] /= image_width  # x
            bbox[1] /= image_height # y
            bbox[2] /= image_width  # width
            bbox[3] /= image_height # height
            bbox_str = ' '.join(map(str, bbox))

            # Normalize segmentation points
            segmentation = annotation['segmentation'][0]
            segmentation = np.array(segmentation).reshape((-1, 2))
            segmentation[:, 0] /= image_width   # x points
            segmentation[:, 1] /= image_height  # y points
            segmentation_str = ' '.join(map(str, segmentation.flatten()))

            f.write(f"{category_id} {bbox_str} {segmentation_str}\n")


In [4]:
# Paths to your data
coco_annotations_path = 'SAF22_COCO_CorentinDoche.json'
images_dir = 'images'  # Change this to your image directory
output_dir = 'output'  # Directory where YOLO formatted data will be saved

convert_coco_to_yolo(coco_annotations_path, images_dir, output_dir)

In [5]:
from ultralytics import YOLO

In [6]:
model = YOLO("yolov8n-seg.pt")  # load a pretrained model (recommended for training)

In [ ]:
# Train the model
#results = model.train(data="C:/Users/devan/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset.yaml", epochs=50, imgsz=640)
results = model.train(data="C:/Users/ds678/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset_saf22.yaml", epochs=50, imgsz=640)


Ultralytics YOLOv8.2.58  Python-3.11.4 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i9-12900)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/Users/ds678/OneDrive - University of Sussex/SAF_Project/Instance_seg_labels/custom_dataset_saf22.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

100%|██████████| 755k/755k [00:00<00:00, 34.0MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

YOLOv8n-seg summary: 261 layers, 3,264,396 parameters, 3,264,380 gradients, 12.1 GFLOPs

Transferred 381/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning C:\Users\ds678\OneDrive - University of Sussex\SAF_Project\Instance_seg_labels\output\labels\train... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:07<00:00,  7.40it/s]

...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      26/50         0G      1.269     0.8182      1.821       1.75        332        640: 100%|██████████| 4/4 [00:54<00:00, 13.74s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:18<00:00,  9.31s/it]

                   all         53       1524       0.39      0.437      0.374      0.191     0.0565      0.139     0.0535      0.011



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      27/50         0G      1.252      0.856      1.808      1.744       1010        640:  50%|█████     | 2/4 [00:31<00:31, 15.70s/it]

In [ ]:
print(results.class_result(0))
print(results.class_result(1))
print(results.class_result(2))
print(results.class_result(3))